## Comparisons for Adam, AdamW, LAMB

In [1]:
# -*- coding: utf-8 -*-
"""Mnist Classifier Using Tensorflow.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1M0Xh1wX5mfqE9Y22VJoa6o61vhFs-OHY

This Code is to build a classifier for MNIST dataset in Tensorflow
"""

# import all packages on top 
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from optimization import AdamWeightDecayOptimizer, LAMBOptimizer

tf.set_random_seed(12345)


"""one is on and rest are off , used for multiclass classification for one class output will be 1, and for others it be zero"""

# Read and create dataset 

mnist = input_data.read_data_sets('/tmp/data',one_hot=True)

# Define model variables 
n_nodes_hl1 = 256
n_nodes_hl2 = 256
n_nodes_hl3 = 256

n_classes = 10
batch_size = 256

# Place holder to store values while passing through graph 
# float value, Height of row is None and width will be 784
x = tf.placeholder('float',[None,784])
y = tf.placeholder('float')


keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Define MLP

In [0]:
# Method to define model 
def mlp(data):
  # (input_data * weight)  + bias
  hidden_1_layer = {'weights': tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
  hidden_2_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
  hidden_3_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
  output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases': tf.Variable(tf.random_normal([n_classes]))}

  #input_data * weight +b
  l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
  l1 = tf.nn.relu(l1)

  l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']),hidden_2_layer['biases'])
  l2 = tf.nn.relu(l2)

  l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']),hidden_3_layer['biases'])
  l3 = tf.nn.relu(l3)

  output = tf.matmul(l3,output_layer['weights'])+  output_layer['biases']

  return output

### Define CNN

In [0]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#
def cnn(x):
  weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
             'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
             'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),
             'out':tf.Variable(tf.random_normal([1024, n_classes]))}

  biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
             'b_conv2':tf.Variable(tf.random_normal([64])),
             'b_fc':tf.Variable(tf.random_normal([1024])),
             'out':tf.Variable(tf.random_normal([n_classes]))}

  x = tf.reshape(x, shape=[-1, 28, 28, 1])

  conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
  conv1 = maxpool2d(conv1)
  
  conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
  conv2 = maxpool2d(conv2)

  fc = tf.reshape(conv2,[-1, 7*7*64])
  fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
  fc = tf.nn.dropout(fc, keep_rate)

  output = tf.matmul(fc, weights['out'])+biases['out']

  return output

### Define Bi-RNN

In [0]:
def birnn(x, cell_type=tf.nn.rnn_cell.LSTMCell):
  timesteps = 28
  x = tf.reshape(x, [-1, timesteps ,timesteps])
  # Define weights
  weights = {
      # Hidden layer weights => 2*n_hidden because of forward + backward cells
      'out': tf.Variable(tf.random_normal([2*128, 10]))
  }
  biases = {
      'out': tf.Variable(tf.random_normal([10]))
  }
  # Prepare data shape to match `rnn` function requirements
  # Current data input shape: (batch_size, timesteps, n_input)
  # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

  # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
  x = tf.unstack(x, timesteps, 1)

  # Define rnn cells with tensorflow
  fw_cell = cell_type(128)
  bw_cell = cell_type(128)

  # Get cell output
  try:
      outputs, _, _ = tf.nn.static_bidirectional_rnn(fw_cell, bw_cell, x,
                                            dtype=tf.float32)
  except Exception: # Old TensorFlow version only returns outputs not states
      outputs = tf.nn.static_bidirectional_rnn(fw_cell, bw_cell, x,
                                      dtype=tf.float32)

  # Linear activation, using rnn inner loop last output
  output = tf.matmul(outputs[-1], weights['out']) + biases['out']

  return output

### Define Training

In [0]:
# Method to intialize model and train using mnist data and test model function
def run_training(neural_net, cell_type=None):
  print("-"*50)
  print("Current Neural Network: ", neural_net)
  if cell_type is not None:
    prediction = neural_net(x, cell_type)
  else:
    prediction = neural_net(x)
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

  # Adam optimizer does has learning rate parameter
  optimizers = [0,0,0]
  optimizers[0] = tf.train.AdamOptimizer(0.001)
  optimizers[1] = AdamWeightDecayOptimizer(0.001)
  optimizers[2] = LAMBOptimizer(0.001)

  #cycles for feed forward and backprop error
  nm_epochs = 100
  
  #
  for opt in optimizers:
    print("Current Optimizer: ", opt)
    optimizer = opt.minimize(cost)
    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      for epoch in range(nm_epochs):
        epoch_loss = 0
        for _ in range(int(mnist.train.num_examples/batch_size)):
          epoch_x,epoch_y = mnist.train.next_batch(batch_size)
          _ ,c = sess.run([optimizer, cost], feed_dict= {x:epoch_x, y:epoch_y})
          epoch_loss += c
        if epoch % 10 == 0:
          print("Epoch", epoch,' completed out of ', nm_epochs , "Loss:" , epoch_loss)
      correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
      accuracy =  tf.reduce_mean(tf.cast(correct, 'float'))
      print("Accuracuy" ,accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [6]:
run_training(mlp)

--------------------------------------------------
('Current Neural Network: ', <function mlp at 0x7f47d0476938>)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

('Current Optimizer: ', <tensorflow.python.training.adam.AdamOptimizer object at 0x7f47cf877350>)
('Epoch', 0, ' completed out of ', 100, 'Loss:', 462588.7238769531)
('Epoch', 10, ' completed out of ', 100, 'Loss:', 9729.408418655396)
('Epoch', 20, ' completed out of ', 100, 'Loss:', 738.5969310670625)
('Epoch', 30, ' completed out of ', 100, 'Loss:', 459.3601968849771)
('Epoch', 40, ' completed out of ', 100, 'Loss:', 471.4661360885366)
('Epoch', 50, ' completed out of ', 100, 'Loss:', 379.70835684174835)
('Epoch', 60, ' completed out of ', 100, 'Loss:', 279.8096264439516)
('Epoch', 70, ' completed out of ', 100, '

In [7]:
run_training(cnn)

--------------------------------------------------
('Current Neural Network: ', <function cnn at 0x7f47d0476848>)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
('Current Optimizer: ', <tensorflow.python.training.adam.AdamOptimizer object at 0x7f47c1101510>)
('Epoch', 0, ' completed out of ', 100, 'Loss:', 1790345.1608886719)
('Epoch', 10, ' completed out of ', 100, 'Loss:', 19208.704095840454)
('Epoch', 20, ' completed out of ', 100, 'Loss:', 4100.823376274595)
('Epoch', 30, ' completed out of ', 100, 'Loss:', 1961.4625253677368)
('Epoch', 40, ' completed out of ', 100, 'Loss:', 1414.5663056151243)
('Epoch', 50, ' completed out of ', 100, 'Loss:', 1202.5498754279688)
('Epoch', 60, ' completed out of ', 100, 'Loss:', 921.100043296814)
('Epoch', 70, ' completed out of ', 100, 'Loss:', 674.768424814567)
('Epoch', 80, ' completed out of ', 100, 'Loss:', 463.47949127200974)
('Epoch', 90, ' completed out of ', 100, 'Loss:',

In [8]:
run_training(birnn, cell_type=tf.nn.rnn_cell.BasicRNNCell)

--------------------------------------------------
('Current Neural Network: ', <function birnn at 0x7f47d0476d70>)
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell, unroll=True))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
('Current Optimizer: ', <tensorflow.python.training.adam.AdamOptimizer object at 0x7f47c0932a90>)
('Epoch', 0, ' completed out of ', 100, 'Loss:', 136.50434014201164)
('Epoch', 10, ' completed out of ', 100, 'Loss:', 18.987429643049836)
('Epoch', 20, ' completed out of ', 100, 'Loss:', 13.061466319486499)
('Epoch', 30, ' completed out of ', 100, 'Loss:', 10.232898156158626)
('Epoch', 40, ' completed out of ', 100, 'Loss:', 7.890659255674109)
('Epoch', 50, ' completed out of ', 100, 'Loss:', 6.273008

In [9]:
run_training(birnn, cell_type=tf.nn.rnn_cell.GRUCell)

--------------------------------------------------
('Current Neural Network: ', <function birnn at 0x7f47d0476d70>)
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
('Current Optimizer: ', <tensorflow.python.training.adam.AdamOptimizer object at 0x7f482b2f2710>)
('Epoch', 0, ' completed out of ', 100, 'Loss:', 155.9565221220255)
('Epoch', 10, ' completed out of ', 100, 'Loss:', 6.130548636429012)
('Epoch', 20, ' completed out of ', 100, 'Loss:', 2.336274054774549)
('Epoch', 30, ' completed out of ', 100, 'Loss:', 1.1914153546094894)
('Epoch', 40, ' completed out of ', 100, 'Loss:', 0.43508785866106336)
('Epoch', 50, ' completed out of ', 100, 'Loss:', 2.353516362994924)
('Epoch', 60, ' completed out of ', 100, 'Loss:', 0.35459679920313647)
('Epoch', 70, ' completed out of ', 100, 'Loss:', 0.00540988779835061)
('Epoch', 80, ' completed out of ', 100, 'Loss:', 0.0019388693119992695)
('Epoch', 90, ' completed o

In [10]:
run_training(birnn, cell_type=tf.nn.rnn_cell.LSTMCell)

--------------------------------------------------
('Current Neural Network: ', <function birnn at 0x7f47d0476d70>)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
('Current Optimizer: ', <tensorflow.python.training.adam.AdamOptimizer object at 0x7f45e4a4f250>)
('Epoch', 0, ' completed out of ', 100, 'Loss:', 136.01194302737713)
('Epoch', 10, ' completed out of ', 100, 'Loss:', 6.592356271110475)
('Epoch', 20, ' completed out of ', 100, 'Loss:', 3.2185868460219353)
('Epoch', 30, ' completed out of ', 100, 'Loss:', 1.2057269491488114)
('Epoch', 40, ' completed out of ', 100, 'Loss:', 1.0393099158100085)
('Epoch', 50, ' completed out of ', 100, 'Loss:', 1.2134482465698966)
('Epoch', 60, ' completed out of ', 100, 'Loss:', 0.7256674864838715)
('Epoch', 70, ' completed out of ', 100, 'Loss:', 0.7495522389508551)
('Epoch', 80, ' completed out of ', 100, 'Loss:', 0.00742054059264774)
('Epoch', 90, ' completed ou